In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.chrome.options import Options

option = Options()
option.add_argument('--headless')
driver = webdriver.Chrome(options=option)
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
url = 'https://www.drugs.com/sfx/'
driver.get(url)

se_list = []

#この範囲は変更しない
elem_range = driver.find_elements_by_class_name('ddc-clearfix')[1]

for elem in elem_range.find_elements_by_tag_name('li'):
    driver.execute_script("arguments[0].scrollIntoView();", elem)
    sleep(0.9)
    atag1 = elem.find_element_by_tag_name('a')
    new_url = atag1.get_attribute('href')

    driver.execute_script("window.open()")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(new_url)
    sleep(0.9)

    #二つ目のタブ
    dg_range = driver.find_element_by_class_name('ddc-list-column-2')
    for dg_elem in dg_range.find_elements_by_tag_name('li'):
        driver.execute_script("arguments[0].scrollIntoView();", dg_elem)
        sleep(0.9)

        atag2 = dg_elem.find_element_by_tag_name('a')
        dn = atag2.text
        ltst_url = atag2.get_attribute('href')

        #三つ目のタブ
        driver.execute_script("window.open()")
        driver.switch_to.window(driver.window_handles[2])
        driver.get(ltst_url)
        sleep(0.9)

        er1 = driver.find_element_by_class_name('contentBox')

        try:
            if er1.find_element_by_class_name('drug-subtitle'):
                gnr = er1.find_element_by_class_name('drug-subtitle')
                gn = gnr.find_element_by_tag_name('i').text
        except:
            gn = ('none')


        dlt_lst = []
        try:
            if driver.find_element_by_css_selector("#content > div.contentBox > ul.ddc-anchor-links"):
                dlt_elem_rng = driver.find_element_by_css_selector("#content > div.contentBox > ul.ddc-anchor-links")
                for dlt_elem in dlt_elem_rng.find_elements_by_tag_name('li'):
                    dlt_elem_nm = dlt_elem.text
                    dlt_lst = [dn, dlt_elem_nm, gn]
                    se_list.append(dlt_lst)        
        except:
            pass

        dlt_lst2 = []
        try:
            if er1.find_elements_by_id('moreResources'):
                dlt_rng = driver.find_element_by_id('moreResources')
                for dlt_li in dlt_rng.find_elements_by_tag_name('li'):
                    dl = dlt_li.text
                    dlt_lst2 = [dn, dl, gn]
                    se_list.append(dlt_lst2)
        except:
            pass
            

        dg_selst = []
        try:
            for sdefct in er1.find_elements_by_tag_name('li'):
                sdefct_name = sdefct.text
                dg_selst = [dn, sdefct_name, gn]
                se_list.append(dg_selst)
            
        except:
            pass

        driver.close()
        driver.switch_to.window(driver.window_handles[1])
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
driver.quit()

df = pd.DataFrame(se_list, columns=['drug_name', 'side_effect', 'generic_name'])
dfx = df.drop_duplicates(keep=False)
dfx.to_csv('drugSE-2021-12-10-3.csv', index=False)
df.to_csv('drugSE-1210-4.csv', index=False)

In [6]:
#se_list

In [8]:
df = pd.DataFrame(se_list, columns=['drug_name', 'side_effect', 'generic_name'])
dfx = df.drop_duplicates(keep=False)
dfx.to_csv('drugSE-1210.csv', index=False)

In [9]:
df.to_csv('drugSE-1210-2.csv', index=False)

In [11]:
driver.quit()